In [10]:
import os
os.chdir(r'c:\Users\Ken\Documents\GitHub\coursera_rag')


In [12]:
import pandas as pd
df = pd.read_csv('requirements_from_acts.csv')
# df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.to_dict('records')
df

,ID,Document ID,Document Title,Annex,Section L1,Section L2,Section L3,Section L4,Essential Requirement
0,1,2014/30/EU,EMC Directive,NaN,1,NaN,NaN,NaN,General requirements\n\nEquipment shall be so ...
1,2,2014/30/EU,EMC Directive,NaN,2,NaN,NaN,NaN,Specific requirements for fixed installations\...
2,3,2014/35/EU,Low Voltage Directive,I,1,NaN,NaN,NaN,General conditions\n(a) the essential characte...
3,4,2014/35/EU,Low Voltage Directive,I,2,NaN,NaN,NaN,Protection against hazards arising from the el...
4,5,2014/35/EU,Low Voltage Directive,I,3,NaN,NaN,NaN,Protection against hazards which may be caused...
...,...,...,...,...,...,...,...,...,...
222,229,2006/42/EC,Machinery Directive,I,6,4.0,NaN,NaN,6.4. MACHINERY SERVING FIXED LANDINGS
223,230,2006/42/EC,Machinery Directive,I,6,4.0,1.0,NaN,6.4.1. Risks to persons in or on the carrier T...
224,231,2006/42/EC,Machinery Directive,I,6,4.0,2.0,NaN,"6.4.2. Controls at landings Controls, other th..."
225,232,2006/42/EC,Machinery Directive,I,6,4.0,3.0,NaN,6.4.3. Access to the carrier The guards at the...
